In [312]:
import pandas as pd
import numpy as np
import os
pd.options.mode.chained_assignment = None

In [313]:
DATA_DIR = 'data'
city = 'Melbourne'
CITY_DIR = os.path.join(DATA_DIR, city) 
RAW_DIR = os.path.join(CITY_DIR, 'raw')
RAW_CRASH_DIR = os.path.join(RAW_DIR, 'crash')
PROCESSED_CRASH_DIR = os.path.join(CITY_DIR, 'processed', 'crash')
PROCESSED_MAPPING_DIR = os.path.join(CITY_DIR, 'processed', 'mapping')

## Get file names and read in csv's

In [314]:
crash_file = os.path.join(RAW_CRASH_DIR, 'crash.csv')
map_file = os.path.join(RAW_CRASH_DIR, 'map.csv')
map_inters_file = os.path.join(RAW_CRASH_DIR, 'map_inters.csv')
atmosphere_file = os.path.join(RAW_CRASH_DIR, 'atmosphere.csv')

In [315]:
crash_df = pd.read_csv(crash_file)
map_df = pd.read_csv(map_file)
map_inters_df = pd.read_csv(map_inters_file)
atmosphere_df = pd.read_csv(atmosphere_file)

## crash_df: drop unwanted columns and establish mappings

In [316]:
crash_df_cols = list(crash_df.columns.values)
crash_df_cols_reduced = ['ACCIDENT_NO','ACCIDENTDATE','ACCIDENTTIME','DAY_OF_WEEK','LIGHT_CONDITION','NODE_ID',
                      'ROAD_GEOMETRY','SPEED_ZONE']
geom_mapping_cols = ['ROAD_GEOMETRY', 'Road Geometry Desc']
accident_type_mapping_cols = ['ACCIDENT_TYPE', 'Accident Type Desc']
DCA_code_mapping_cols = ['DCA_CODE', 'DCA Description']
light_condition_mapping_cols = ['LIGHT_CONDITION', 'Light Condition Desc']

In [317]:
geom_mapping = crash_df[geom_mapping_cols]
geom_mapping = geom_mapping.drop_duplicates().sort_values(by='ROAD_GEOMETRY').reset_index(drop=True)

accident_type_mapping = crash_df[accident_type_mapping_cols]
accident_type_mapping = accident_type_mapping.drop_duplicates().sort_values(by='ACCIDENT_TYPE').reset_index(drop=True)

light_condition_mapping = crash_df[light_condition_mapping_cols]
light_condition_mapping = light_condition_mapping.drop_duplicates().sort_values(by='LIGHT_CONDITION').reset_index(drop=True)

DCA_code_mapping = crash_df[DCA_code_mapping_cols]
DCA_code_mapping = DCA_code_mapping.drop_duplicates().sort_values(by="DCA_CODE").reset_index(drop=True)

In [318]:
crash_df_reduced = crash_df[crash_df_cols_reduced]

## map_df: drop unwanted columns, create node type mapping

In [319]:
node_type_mapping = pd.DataFrame({'NODE_TYPE_INT': [0, 1, 2, 3], 'NODE_TYPE': ['I', 'N', 'O', 'U'], 
                                  'NODE_DESC': ['Intersection', 'Non-intersection', 'Off-road', 'Unknown']})

In [320]:
map_df['NODE_TYPE_INT'] = ""
for index, row in node_type_mapping.iterrows():
    map_df['NODE_TYPE_INT'].loc[map_df['NODE_TYPE'] == row['NODE_TYPE']]  = row['NODE_TYPE_INT']

In [321]:
map_df_reduced_cols = ['ACCIDENT_NO', 'NODE_ID', 'NODE_TYPE_INT', 'LGA_NAME', 'Deg Urban Name', 'Lat', 'Long']
map_df_reduced = map_df[map_df_reduced_cols]

## map_inters_df: drop unwanted columns, create node to complex node mapping

In [322]:
complex_node_mapping_cols = ['NODE_ID', 'COMPLEX_INT_NO']
complex_node_mapping = map_inters_df[complex_node_mapping_cols]
complex_node_mapping = complex_node_mapping.drop_duplicates().sort_values(by="NODE_ID").reset_index(drop=True)

In [323]:
map_inters_df_reduced = map_inters_df[['ACCIDENT_NO', 'COMPLEX_INT_NO']]

## atmosphere_df: drop unwanted columns, create atmosphere mapping

In [324]:
atmosphere_mapping_cols = ['ATMOSPH_COND', 'Atmosph Cond Desc']
atmosphere_mapping = atmosphere_df[atmosphere_mapping_cols]
atmosphere_mapping = atmosphere_mapping.drop_duplicates().sort_values(by="ATMOSPH_COND").reset_index(drop=True)

In [325]:
atmosphere_df_reduced_cols = ['ACCIDENT_NO', 'ATMOSPH_COND']
atmosphere_df_reduced = atmosphere_df[atmosphere_df_reduced_cols]

## Drop duplicates from all dataframes

In [326]:
crash_df_reduced.drop_duplicates(subset="ACCIDENT_NO", inplace=True)
map_df_reduced.drop_duplicates(subset="ACCIDENT_NO", inplace=True)
map_inters_df_reduced.drop_duplicates(subset="ACCIDENT_NO", inplace=True)
atmosphere_df_reduced.drop_duplicates(subset="ACCIDENT_NO", inplace=True)

## crashes_df: join all of above dataframes on 'accident_no'

## crashes_df: cleanup, add derived features, rename, validate data

In [334]:
crashes_df.head()

,ACCIDENT_NO,ACCIDENT_DATE,ACCIDENT_TIME,DAY_OF_WEEK,LIGHT_CONDITION,NODE_ID,ROAD_GEOMETRY,SPEED_ZONE,ATMOSPH_COND,NODE_TYPE_INT,SUBURB,DEGREE_URBAN,LAT,LON,COMPLEX_INT_NO,COMPLEX_NODE
0,T20060000010,13/01/2006,12.42.00,6,1,43078,1,60,1,0,DANDENONG,MELB_URBAN,-37.98862,145.21805,0,0
1,T20060000018,13/01/2006,19.10.00,6,1,29720,2,70,1,1,CASEY,MELB_URBAN,-37.99092,145.27632,0,0
2,T20060000022,14/01/2006,12.10.00,7,1,203074,5,100,1,1,MORNINGTON PENINSULA,RURAL_VICTORIA,-38.39632,144.85489,0,0
3,T20060000023,14/01/2006,11.49.00,7,1,55462,2,80,1,0,DANDENONG,MELB_URBAN,-37.98918,145.14496,0,0
4,T20060000026,14/01/2006,10.45.00,7,1,202988,5,50,1,1,MORNINGTON PENINSULA,MELB_URBAN,-38.37299,144.87159,0,0


In [328]:
crashes_df.dropna(subset=['Lat', 'Long'], inplace = True)
crashes_df.drop(['NODE_ID_y'], axis = 1, inplace = True)

In [330]:
crashes_df.fillna(value=0, inplace=True)
crashes_df[['NODE_ID_x', 'COMPLEX_INT_NO']] = crashes_df[['NODE_ID_x', 'COMPLEX_INT_NO']].astype(int)

In [331]:
crashes_df['COMPLEX_NODE'] = 0
crashes_df['COMPLEX_NODE'].loc[crashes_df['COMPLEX_INT_NO'] > 0]  = 1

In [353]:
# NOTE USE THIS CELL IF BY THE HOUR IS NOT GRANULAR ENOUGH

def mapTime(x):
    hour = x[:2]
    minute = float(x[3:5])
    if minute < 30:
        minute = 0
    else:
        minute = 0.5
    time = float(hour) + minute
    return time

crashes_df['HOUR'] = crashes_df['ACCIDENT_TIME'].apply(mapTime)
crashes_df.head()

In [358]:
crashes_df['HOUR'] = crashes_df['ACCIDENT_TIME'].apply(lambda x: x[:2])

In [359]:
crashes_df.rename(columns={"NODE_ID_x":"NODE_ID", "ACCIDENTDATE":"ACCIDENT_DATE", "ACCIDENTTIME":"ACCIDENT_TIME", 
                           "LGA_NAME":"SUBURB", "Deg Urban Name":"DEGREE_URBAN", "Lat":"LAT", "Long":"LON"}, inplace=True)

## Output crashes_df and all mappings

In [360]:
if not os.path.exists(PROCESSED_CRASH_DIR):
    os.makedirs(PROCESSED_CRASH_DIR)

if not os.path.exists(PROCESSED_MAPPING_DIR):
    os.makedirs(PROCESSED_MAPPING_DIR)
    
crashes_path = os.path.join(PROCESS_CRASH_DIR, 'crashes.csv')
crashes_df.to_csv(crashes_path)
    
mapping_dfs = [geom_mapping, accident_type_mapping, DCA_code_mapping, 
               light_condition_mapping, node_type_mapping, atmosphere_mapping]
mapping_names = ['geom_mapping.csv', 'accident_type_mapping.csv', 'DCA_code_mapping.csv', 
               'light_condition_mapping.csv', 'node_type_mapping.csv', 'atmosphere_mapping.csv']

for mapping_df, mapping_name in zip(mapping_dfs, mapping_names):
    save_path = os.path.join(PROCESSED_MAPPING_DIR, mapping_name)
    mapping_df.to_csv(save_path)

In [370]:
my_df = pd.DataFrame({'Hi': [1,2,3], 'bob': ['a','b', 'c'], 'friend': ['x', 'y', 'z']})

In [371]:
my_df.set_index('Hi', inplace=True)

In [372]:
my_df

,bob,friend
Hi,,
1,a,x
2,b,y
3,c,z


In [374]:
my_df.to_json(orient='index')

'{"1":{"bob":"a","friend":"x"},"2":{"bob":"b","friend":"y"},"3":{"bob":"c","friend":"z"}}'